In [ ]:
# Some jupyter notebook magic to reload modules automatically when they change
# not necessary for this specific notebook but useful in general
%load_ext autoreload
%autoreload 2

In [ ]:
from definitions import ROOT_DIR
import os

# The PSDM specific input models can be imported from the pypsdm.models.input and
# pypsdm.models.result. The `GridWithResults` container is located in pypsdm.models.gwr
from pypsdm.models.gwr import GridWithResults


grid_path = os.path.join(ROOT_DIR, "tests", "resources", "simbench", "input")
result_path = os.path.join(ROOT_DIR, "tests", "resources", "simbench", "results")
# IO data models in general have a from_csv method to parse psdm files
gwr = GridWithResults.from_csv(grid_path, result_path)

In [ ]:
from pypsdm.plots.grid import grid_plot

# Use the grid_plot method to visualize the grid model
# only works if the underlying node input files have associated coordinates
grid_plot(gwr.grid)

In [ ]:
results = gwr.results
# The grid results are symmetrical to the input grid, so there is a result container
# for participants and for the raw grid.
raw_grid_res = results.raw_grid
participants_res = results.participants

In [ ]:
# Results for each grid element and participant is a discrete event time series
# which basically means each recorded state of e.g. a node (consisting of current magnitude, and angle)
# is valid until the next recorded state.

nodes_res = raw_grid_res.nodes
# Reminder: you can also access the nodes result directly from the gwr e.g. gwr.nodes_res
nodes_res = gwr.nodes_res

In [ ]:
# The underlying structure of the result data is a dictionary subclass
# the values of the dictionary are dependent on the result model
# e.g. the result dictionary NodesResult contains ComplexVoltage objects


# You can access items as you would with a standard dictionary
uuid = list(nodes_res.keys())[0]
node_res = nodes_res[uuid]

# Similar to the input models the underlying data structure of each singular result object
# is a pandas DataFrame with a date time index
node_res.data.head()

In [ ]:
from datetime import datetime


# If you want to look at some time interval, you can slice the result models with date
# time objects
start = datetime(2016, 1, 1)
end = datetime(2016, 1, 1, 1)
sliced_node_res = node_res[start:end]
sliced_node_res.data

In [ ]:
from pypsdm.models.result.grid.node import NodesResult


# You can also filter the whole dictionary for a time interval
sliced_nodes_res = nodes_res.interval(start, end)
sliced_nodes_res[uuid].data

In [ ]:
# For participants all things look mostly the same
# The type of results are mostly of type PQResult, containing active and reactive power


loads_res = participants_res.loads
load_res = loads_res[list(loads_res.keys())[0]]
load_res.data.head()

In [ ]:
# There are calculation utilities implemented where it makes sense
load_res2 = loads_res[list(loads_res.keys())[1]]
load_sum = load_res + load_res2
load_sum.data.head()

In [ ]:
# You can also calculate the sum for all participants
total_loads_res = loads_res.sum()
total_loads_res.data.head()

In [ ]:
# If you care about a specific attribute of all entities in a result dict you can assemble
# a data frame with all time series of all entities in the dict
loads_res_p = loads_res.p().head()
loads_res_p

In [ ]:
# By default the data frame uses the ids of the loads if available instead of the uuids.
# You can change this behavior with the favor_ids argument
loads_res_p = loads_res.p(favor_ids=False).head()
loads_res_p

## Nodal Results

Often times we care about nodal results, so the net load and generation at individual
nodes. Therefore, we include a special data model `ExtendedNodesResult` which is
calculated by summing up the results of all connected participants at the individual nodes.


In [ ]:
# There is a method to calculate all of them
nodal_results = gwr.build_extended_nodes_result()

target_uuid = list(nodal_results.keys())[3]
nodal_results[target_uuid].data.head()

In [ ]:
# If you care about the individual systems at each node, you can filter down the
# result container to the systems connected to a specified node

nodal_gwr = gwr.nodal_result(target_uuid)
nodal_gwr.loads.p()

In [ ]:
# Let's say you want to make sure that all the load is actually connected to the
# node that we have filtered for
load_uuids = list(nodal_gwr.loads.keys())

# You can get a subset of the input data model via a list of uuids
# Note that the `node` attribute confirms that indeed the load is connected
# to the node we have filtered for
gwr.loads.subset(load_uuids[0]).data

## Line Results

Here is some example for line results to analyse for line ratings etc.

In [ ]:
# If you care about the individual line element, you can filter down the
# result container to the systems connected to a specified line

line_uuid = gwr.lines.data.iloc[0].name
line_gwr = gwr.lines_res[line_uuid]
line_gwr.data.head()

In [ ]:
# If we would like to get the utilisation of the line segment, we need use i_max provide from input data (gwr.lines)

line_input_data = gwr.lines
line_utilization = gwr.lines_res.utilisation(line_input_data, side="a")
line_utilization.head()

In [ ]:
line_utilization[[line_uuid]].head()

For more details please check the implemented methods of all the data models

Check out the `docs/nbs/plots.ipynb` notebook for some examples of the included plotting utilities


## Congestion Results

If we want to analyze congestions in the grid, we can run SIMONA with congestion detection. All congestion results are mapped uuid of the asset for which the congestion was detected. The type information (e.g.: node, line, ect.) specifies type of the asset that has a congestion.

Each congestion contains the value that occurred and the limits for the asset (e.g.: voltage band, line current limit, etc.).

In [ ]:
# Congestion results are only provided for two nodes
node_with_congestions = [
    "5d50a881-c383-463e-8355-41b3dd57422d",
    "557b9f51-d83c-476c-a84c-d240530c203d",
]

congestion_res_1 = gwr.congestions_res[node_with_congestions[0]]
congestion_res_2 = gwr.congestions_res[node_with_congestions[1]]

congestion_res_1.data